## References:   

### Qiskit runtime and VQE:
https://qiskit.org/documentation/nature/tutorials/07_leveraging_qiskit_runtime.html
https://github.com/Qiskit-Partners/qiskit-runtime/blob/main/tutorials/vqe.ipynb 
### QUBO on Qiskit:
https://qiskit.org/documentation/tutorials/optimization/3_minimum_eigen_optimizer.html
https://qiskit.org/documentation/tutorials/optimization/2_converters_for_quadratic_programs.html
https://qiskit.org/documentation/tutorials/optimization/1_quadratic_program.html

In [1]:
from qiskit import BasicAer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)

## Construction of ising hamiltonian

In [3]:
def potential_pseudoknots(stems_potential):

    pseudoknots_potential = {}
    pseudoknot_penalty = 0.5

    for i in range(len(stems_potential)):
        for j in range(i + 1, len(stems_potential)):
            
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            i_a = stem1[0]
            j_a = stem1[1]
            i_b = stem2[0]
            j_b = stem2[1]
    
            penalty = 1
    
            if (i_a < i_b and i_b < j_a and j_a < j_b) or (i_b < i_a and i_a < j_b and j_b < j_a):
        
                penalty = pseudoknot_penalty
    
            pseudoknots_potential[(j,i)] = penalty
            
    return pseudoknots_potential

In [4]:
def potential_overlaps(stems_potential):
    
    overlaps_potential = {}
    overlap_penalty = 1e6

    for i in range(len(stems_potential)):
        for j in range(i+1, len(stems_potential)):
    
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            penalty = 0
    
            stem1_cspan1 = set(range(stem1[1]-stem1[2]+1, stem1[1]+1))
            stem2_cspan1 = set(range(stem2[1]-stem2[2]+1, stem2[1]+1))
            
            stem1_cspan2 = set(range(stem1[0], stem1[0]+stem1[2]))
            stem2_cspan2 = set(range(stem2[0], stem2[0]+stem2[2]))
    
            if (len(stem1_cspan1 & stem2_cspan1) != 0) or (len(stem1_cspan2 & stem2_cspan2) != 0)  or (len(stem1_cspan1 & stem2_cspan2) != 0) or (len(stem1_cspan2 & stem2_cspan1) != 0):
        
                penalty = overlap_penalty
        
            overlaps_potential[(j,i)] = penalty
            
    return overlaps_potential

In [5]:
stems = [(1,14,3),(5,18,3),(9,18,5)] 
CL = 1
CB = 1
mu = 3
pseudoknot = potential_pseudoknots(stems)
overlap = potential_overlaps(stems)
linear_terms = []
quadratic_terms = {}

In [6]:
print(pseudoknot)
print(overlap)

{(1, 0): 0.5, (2, 0): 0.5, (2, 1): 1}
{(1, 0): 0, (2, 0): 1000000.0, (2, 1): 1000000.0}


In [7]:
def calculate_linear_term(i):
    stem = stems[i]
    k = stem[2]
    return CL*(k*k-2*k*mu+mu*mu)-CB*k*k

In [8]:
def calculate_quadratic_term(i,j):
    k_i = stems[i][2]
    k_j = stems[j][2]
    return 2*CB*k_i*k_j*pseudoknot[(i,j)]+overlap[(i,j)]

In [9]:
from qiskit_optimization import QuadraticProgram
qubo = QuadraticProgram()
for i in range(0,len(stems)):
    qubo.binary_var('q'+str(i))
    linear_terms.append(calculate_linear_term(i))
for i in range(0,len(stems)):
    for j in range(0,len(stems)):
        if i>j:
            quadratic_terms[('q'+str(i),'q'+str(j))] = calculate_quadratic_term(i,j)
qubo.minimize(linear=linear_terms, quadratic=quadratic_terms)

In [10]:
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 9 q0 - 9 q1 - 21 q2 + [ 18 q0*q1 + 2000030 q0*q2 + 2000060 q1*q2 ]/2
Subject To

Bounds
 0 <= q0 <= 1
 0 <= q1 <= 1
 0 <= q2 <= 1

Binaries
 q0 q1 q2
End



In [11]:
op, offset = qubo.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: 499994.0
operator:
-500000.75 * ZII
- 250005.25 * IZI
+ 250007.5 * ZZI
- 250001.5 * IIZ
+ 250003.75 * ZIZ
+ 2.25 * IZZ


In [18]:
from qiskit import IBMQ
provider = IBMQ.load_account()
#backend = provider.backend.ibmq_bogota
#backend = provider.get_backend('ibmq_bogota')
#backend = provider.get_backend('ibmq_lima')
#backend = provider.get_backend('ibmq_manila')
backend= BasicAer.get_backend('statevector_simulator')

ibmqfactory.load_account:WARNING:2022-01-10 12:42:29,371: Credentials are already in use. The existing account in the session will be replaced.


In [14]:
algorithm_globals.random_seed = 10598
quantum_instance = QuantumInstance(backend,
                                   seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
qaoa = MinimumEigenOptimizer(qaoa_mes)
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

optimal function value: -21.0
optimal value: [0. 0. 1.]
status: SUCCESS


## Run on Qiskit runtime

In [28]:
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.quantum_info import SparsePauliOp
ok = SparsePauliOp(op.oplist)
nk = PauliSumOp(ok)
op_list = op.oplist
def tuplify(op):
    return (op.primitive,op.coeff)
pauli_list = map(tuplify,op_list)
print(list(pauli_list))

AttributeError: 'PauliSumOp' object has no attribute 'oplist'

In [94]:
from qiskit import IBMQ

options = {
	'backend_name': 'ibmq_qasm_simulator'
}

runtime_inputs = {
	# The fraction of top measurement
  # samples to be used for
  # the expectation value (CVaR expectation).
  # Defaults to 1, i.e. using
  # all samples to construct the
  # expectation value.
  'alpha': None, # number
	
	# A list of operators to
  # be evaluated at the final,
  # optimized state. This must be
  # a List[PauliSumOp].
  'aux_operators': None, # array
	
	# Initial parameters of the ansatz.
  # Can be an array or
  # the string ``'random'`` to choose
  # random initial parameters. The type
  # must be numpy.ndarray or str.
  'initial_point': None, # [array,string]
	
	# Whether to apply measurement error
  # mitigation in form of a
  # tensored measurement fitter to the
  # measurements. Defaults to False.
  'measurement_error_mitigation': None, # boolean
	
	# The cost Hamiltonian, consisting of
  # Pauli I and Z operators,
  # whose smallest eigenvalue we're trying
  # to find. The type must
  # be a PauliSumOp.
  'operator': ok, # object (required)
	
	# The optimization level to run
  # if the swap strategies are
  # not used. This value is
  # 1 by default. This is
  # an integer.
  'optimization_level': None, # integer
	
	# The classical optimizer used to
  # update the parameters in each
  # iteration. Per default, SPSA with
  # automatic calibration of the learning
  # rate is used. The type
  # must be a qiskit.algorithms.optimizers.Optimizer.
  'optimizer': None, # object
	
	# The number of QAOA repetitions,
  # i.e. the QAOA depth typically
  # labeled p. This value defaults
  # to 1. This is an
  # integer.
  'reps': None, # integer
	
	# The integer number of shots
  # used for each circuit evaluation.
  # Defaults to 1024.
  'shots': None, # integer
	
	# A boolean flag that, if
  # set to True (the default
  # is False), runs a heuristic
  # algorithm to permute the Paulis
  # in the cost operator to
  # better fit the coupling map
  # and the swap strategy. This
  # is only needed when the
  # optimization problem is sparse and
  # when using swap strategies to
  # transpile.
  'use_initial_mapping': None, # boolean
	
	# A boolean on whether or
  # not to use a pulse-efficient
  # transpilation. This flag is set
  # to False by default.
  'use_pulse_efficient': None, # boolean
	
	# A boolean on whether or
  # not to use swap strategies
  # when transpiling. This flag is
  # set to True by default.
  # If this is False then
  # the standard transpiler with the
  # given optimization level will run.
  'use_swap_strategies': None # boolean
}

IBMQ.load_account()
provider = IBMQ.get_provider(
	hub='ibm-q',
	group='open',
	project='main'
)

job = provider.runtime.run(
	program_id='qaoa',
	options=options,
	inputs=runtime_inputs
)

# Job id
print(job.job_id())
# See job status
print(job.status())

# Get results
result = job.result()

ibmqfactory.load_account:WARNING:2022-01-04 15:38:11,611: Credentials are already in use. The existing account in the session will be replaced.


RequestsApiError: '400 Client Error: Bad Request for url: https://runtime-us-east.quantum-computing.ibm.com/jobs. {"errors":[{"code":"bad_request","message":"models.ProgramRunParams.Params: ReadMapCB: expect { or n, but found [, error found in #10 byte of ...|params\\": [\\"{\\\\\\"alpha\\\\|..., bigger context ...|in\\", \\"backend\\": \\"ibmq_qasm_simulator\\", \\"params\\": [\\"{\\\\\\"alpha\\\\\\": null, \\\\\\"aux_operators\\\\\\": null, \\\\\\"init|...","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"c7ab0ed6m4ar19kj27mg"}'

In [95]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement="linear", insert_barriers=True)
ansatz.decompose().draw("mpl", style="iqx")
from qiskit.algorithms.optimizers import SPSA

optimizer = SPSA(maxiter=100)

np.random.seed(5)  # fix seed for reproducibility
initial_point = np.random.random(ansatz.num_parameters)

In [97]:
from qiskit import IBMQ

options = {
	'backend_name': 'ibmq_qasm_simulator'
}

runtime_inputs = {
	# A parameterized quantum circuit preparing
  # the ansatz wavefunction for the
  # VQE. It is assumed that
  # all qubits are initially in
  # the 0 state.
  'ansatz': ansatz, # object (required)
	
	# A list of operators to
  # be evaluated at the final,
  # optimized state. This is a
  # list of PauliSumOp.
  'aux_operators': None, # array
	
	# Initial position of virtual qubits
  # on the physical qubits of
  # the quantum device. Default is
  # None.
  'initial_layout': None, # [null,array,object]
	
	# Initial parameters of the ansatz.
  # Can be an array or
  # the string ``'random'`` to choose
  # random initial parameters.
  'initial_parameters': initial_point, # [array,string] (required)
	
	# Whether to apply measurement error
  # mitigation in form of a
  # complete measurement fitter to the
  # measurements. Defaults to False.
  'measurement_error_mitigation': None, # boolean
	
	# The Hamiltonian whose smallest eigenvalue
  # we're trying to find. Should
  # be PauliSumOp
  'operator': nk, # object (required)
	
	# The classical optimizer used in
  # to update the parameters in
  # each iteration. Can be either
  # any of Qiskit's Optimizer classes.
  # If a dictionary, only SPSA
  # and QN-SPSA are supported and
  # the dictionary must specify the
  # name and options of the
  # optimizer, e.g. ``{'name': 'SPSA', 'maxiter':
  # 100}``.
  'optimizer': optimizer, # object (required)
	
	# The number of shots used
  # for each circuit evaluation. Defaults
  # to 1024.
  'shots': None # integer
}

IBMQ.load_account()
provider = IBMQ.get_provider(
	hub='ibm-q',
	group='open',
	project='main'
)

job = provider.runtime.run(
	program_id='vqe',
	options=options,
	inputs=runtime_inputs
)

ibmqfactory.load_account:WARNING:2022-01-04 15:48:35,695: Credentials are already in use. The existing account in the session will be replaced.


RequestsApiError: '400 Client Error: Bad Request for url: https://runtime-us-east.quantum-computing.ibm.com/jobs. {"errors":[{"code":"bad_request","message":"models.ProgramRunParams.Params: ReadMapCB: expect { or n, but found [, error found in #10 byte of ...|params\\": [\\"{\\\\\\"ansatz|..., bigger context ...|in\\", \\"backend\\": \\"ibmq_qasm_simulator\\", \\"params\\": [\\"{\\\\\\"ansatz\\\\\\": {\\\\\\"__type__\\\\\\": \\\\\\"QuantumCircuit\\\\\\", \\\\|...","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"c7ab5a56m4ar19kj31fg"}'